### Calculating Similarity

The inspiration for this approach originates from a reading of this article:

###### BERT For Measuring Text Similarity
[High-performance semantic similarity with BERT](https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1)

Our approach will be to read all the summaries from the articles data base, use them to encode and create embeddings and compare the candidate articles to them, then observe scores.

In [1]:
import sys
import os
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"
module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 


sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


In [2]:
import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import newspaper
from newspaper import Article 
from keybert import KeyBERT

import json 
import boto3 
import logging 
import uuid 
import time
import datetime
from datetime import datetime as DT
from pathlib import Path 
import os, sys 
import glob
import nltk
from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline
import pandas as pd
import numpy as np
import contractions

# nltk.download('punkt')
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path) 

#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"
TRANSFORMERMODEL = 'all-MiniLM-L6-v2'
from dynamoDButils import *
from APIutils import *
from seedterms import *
from utils import *

import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT

def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')

[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Cell imports done @ 2023-02-07 22:01:11


In [3]:
%%time
# Anna , if you want try different transformers?
model = SentenceTransformer(TRANSFORMERMODEL)
complete(state='model initialised')


Cell model initialised @ 2023-02-07 22:01:12
CPU times: user 497 ms, sys: 171 ms, total: 668 ms
Wall time: 1.41 s


Next choose the corpus of news articles 

In [4]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsn2WD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsn2WD.observe(on_change)
display(jsn2WD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [12]:
# iWD.value
jsnf = jsn2WD.value
print (f"{jsnf=} ")

# jsnfile = osp.join (JSONPath,jsnf)
# xposedf = pd.read_json(jsnfile)
# xposedf = xposedf.transpose() # pivot the matrix
# xposedf

jsnf='semanticSimArticles.json' 


#### Read Corpus  articles and create sentence embeddings
Here, we begin the process of looking at a collection of articles and rating them for inclusion vice a model constructed from the existing corpus of articles

In [13]:
jsnfile = osp.join (JSONPath,jsnf)
cSummaries = pd.read_json(jsnfile)
cSummaries = cSummaries.transpose() # pivot the matrix
cSummaries

,url,title,summary
0,https://www.cnn.com/2023/01/16/politics/overcl...,Analysis: There's a larger 'classified' proble...,A version of this story appeared in CNN’s What...
1,https://www.techrepublic.com/article/best-digi...,Best digital collaboration tools for your busi...,These 10 real-time digital collaboration apps ...
2,https://www.businessinsider.com/i-asked-chatgp...,I asked ChatGPT to do my work and write an Ins...,"I asked ChatGPT, a new AI chatbot from OpenAI,..."
3,https://www.engadget.com/netflixs-dog-and-boy-...,Netflix's 'Dog and Boy' anime causes outrage f...,"In 2016, Studio Ghibli co-founder and director..."
4,https://www.macworld.com/article/1484137/apple...,Apple’s next battle in its ‘silent war’ agains...,While we’re all waiting for Apple to unveil it...
...,...,...,...
3539,https://www.cnn.com/2023/01/10/asia/china-fune...,China: Satellite images capture crowding at cr...,CNN —Satellite images taken over a number of C...
3540,https://www.techrepublic.com/article/nrf-2023-...,NRF 2023: How ambient computing could change r...,Wiliot’s cloud-connected tags attempt to solve...
3541,https://www.macworld.com/article/1476747/homep...,"Defying logic, Apple announces 2nd-gen HomePod...",A day after unveiling updated Macs with M2 pro...
3542,https://www.digitaltrends.com/dtdeals/lenovo-l...,Lenovo Legion gaming PCs and laptops are on sa...,"As we’re moving through January, we keep seein..."


In [14]:
cSentences = []
cSentences = cSummaries.summary.to_list()

complete(state="models")
cSentences[0]


Cell models @ 2023-02-07 22:03:24


'A version of this story appeared in CNN’s What Matters newsletter. To get it in your inbox, sign up for free here.CNN —This is getting absurd.Americans have had the opportunity to choose from between three people to be president since 2016, and all three have been found to have classified documents they should not have.Clearly, there’s a larger problem.Here’s what we don’t know:We, the public, don’t know what documents President Joe Biden had locked up in his garage or in the manila folder in his Penn Biden Center office.We don’t know specifics about what documents the FBI photographed on the floor at former President Donald Trump’s Mar a Lago resort.We don’t even know for sure what documents were on Hillary Clinton’s email server, although we do know that dozens of those documents were retroactively classified years after the fact, long after the existence of her private email server became a political albatross.Here’s what we do know:Classified documents, and their appearance in ema

#### Candidates for inclusion
Next read in the candidate collection and use it to create an embedding

In [17]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [18]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnfile = osp.join (JSONPath,jsnf)
xSummaries = pd.read_json(jsnfile)
# xSummaries = xSummaries.transpose() # pivot the matrix
xSummaries

jsnf='semanticSimArticlesTestingDF.json' 


,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/news/spacex-launch-tod...,spacex falcon rocket carrying gps navigation s...,0.8183,0.4993,0.4775,0.4735,0.4719,0.480550,0.531412,0.511708,0.454913,0.436846,0.434590,0.473894,technology,0.496222,innovation,0.111382
121,https://www.cbsnews.com/video/chatgpt-jobs-art...,rise could chatgpt take job chatbots artificia...,0.7697,0.7255,0.7166,0.7103,0.6910,0.710850,0.637201,0.625964,0.625526,0.619846,0.619757,0.622773,technology,0.675250,innovation,0.216986
122,https://www.cbsnews.com/news/ukraine-military-...,jonas ohman founder ceo blue yellow lithuania ...,0.4923,0.4892,0.4892,0.4847,0.4596,0.488850,0.457436,0.457436,0.430608,0.427529,0.421351,0.438872,technology,0.349854,social,0.111065
123,https://www.cbsnews.com/news/metaverse-making-...,metaverse asset digital art video game charact...,0.6600,0.6304,0.6264,0.6229,0.6044,0.626567,0.614686,0.609527,0.597931,0.587996,0.571264,0.596281,games,0.463839,entertainment,0.240798


In [19]:
xSentences = preProcessing(xSummaries)


Cell models @ 2023-02-07 22:04:25


In [20]:
xSentences[0]

'economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look'

### Encode the sentences:

In [21]:
xSentence_embeddings = model.encode(xSentences)
# xSentence_embeddings.shape

In [22]:
cSentence_embeddings = model.encode(cSentences)
# cSentence_embeddings.shape

In [23]:
print (f"cSentence_embeddings: {cSentence_embeddings[0]}", end="\n\n")
print (f"xSentence_embeddings: {xSentence_embeddings[0]}")

cSentence_embeddings: [-4.83780950e-02 -2.75095012e-02  1.60715561e-02  1.57002322e-02
  1.07021168e-01 -6.50019804e-03 -5.05124331e-02 -3.57177258e-02
  3.41106467e-02  1.23618329e-02  1.31691173e-02  7.79866502e-02
  1.63981132e-03 -2.89458707e-02 -9.97683480e-02 -1.36307552e-02
 -4.19064239e-02 -1.08491322e-02 -5.21043949e-02  1.15556374e-01
 -5.46976784e-03  3.24263088e-02  7.18414336e-02 -1.96892284e-02
 -4.54587769e-03  2.31480282e-02 -6.77628741e-02 -4.29963991e-02
 -1.11857548e-01 -3.34735513e-02  6.20625019e-02  6.09268621e-02
  3.69292907e-02  4.23717164e-02  1.08853273e-01 -2.45025735e-02
  3.42690572e-02  2.07658634e-02  1.02707215e-01 -1.64925754e-02
 -3.63139957e-02 -1.46217436e-01 -6.89819129e-03  2.93060448e-02
 -3.81086767e-02  6.14364371e-02 -2.63481401e-02  5.28832376e-02
 -1.92858465e-02 -6.94705471e-02 -5.63247390e-02  5.88743836e-02
  1.37572223e-02  5.57106510e-02 -4.10957448e-02 -4.67587709e-02
 -1.60674006e-02  2.61155833e-02 -2.62322207e-03  6.30843565e-02
  3

In [24]:
def getHiFive(cs):
    similarities = []
    ord         = 0
    for elt in cs:
        ord += 1
        similarities.append([ord, elt])
    s = sorted(similarities,key=lambda x: x[1] )
    s = s[len(s)-6:len(s)-1]
    print (f"S: {s}")
    return s

In [25]:
scores = []
for i in range (0, len(xSentence_embeddings)):
    cs = cosine_similarity(
        [xSentence_embeddings[i]],
        cSentence_embeddings[:]
    )
    s = getHiFive(cs[0])
    scores.append(s)
complete(state="xSentence_embeddings")

S: [[1685, 0.36535823], [2113, 0.38596308], [2972, 0.39575565], [1523, 0.40861467], [10, 0.40892553]]
S: [[102, 0.43175906], [2847, 0.4427842], [2851, 0.44766027], [2020, 0.44854504], [2575, 0.513474]]
S: [[2679, 0.5337487], [2673, 0.53469914], [2917, 0.5449779], [3027, 0.54650414], [162, 0.54942125]]
S: [[439, 0.47718987], [2208, 0.4820389], [695, 0.48899668], [1401, 0.49640256], [1359, 0.5192921]]
S: [[1132, 0.39554137], [123, 0.41979837], [381, 0.42292386], [2780, 0.42731985], [1235, 0.42812175]]
S: [[2998, 0.39316136], [3393, 0.3939251], [525, 0.3984921], [1740, 0.40538704], [424, 0.42545706]]
S: [[2573, 0.5129419], [1469, 0.5232953], [1391, 0.52846324], [2609, 0.55387247], [3073, 0.57499015]]
S: [[1012, 0.24996966], [21, 0.25357333], [772, 0.26002645], [3502, 0.26345384], [2980, 0.26359376]]
S: [[1728, 0.39779788], [2819, 0.4016285], [1420, 0.40684512], [2226, 0.4334748], [2156, 0.45786142]]
S: [[3328, 0.40899372], [3073, 0.41031355], [1623, 0.4132899], [3293, 0.41696042], [691, 0

S: [[1012, 0.24996966], [21, 0.25357333], [772, 0.26002645], [3502, 0.26345384], [2980, 0.26359376]]
S: [[888, 0.5139053], [1731, 0.5521536], [827, 0.55949736], [2971, 0.55949736], [2572, 0.57051]]
S: [[1012, 0.24996966], [21, 0.25357333], [772, 0.26002645], [3502, 0.26345384], [2980, 0.26359376]]
S: [[1012, 0.24996966], [21, 0.25357333], [772, 0.26002645], [3502, 0.26345384], [2980, 0.26359376]]
S: [[1012, 0.24996966], [21, 0.25357333], [772, 0.26002645], [3502, 0.26345384], [2980, 0.26359376]]
S: [[1247, 0.5578319], [2724, 0.58608997], [2446, 0.59442973], [1731, 0.6250254], [1004, 0.67015874]]
S: [[1012, 0.24996966], [21, 0.25357333], [772, 0.26002645], [3502, 0.26345384], [2980, 0.26359376]]
S: [[14, 0.46741107], [80, 0.47007445], [3095, 0.4813618], [736, 0.498898], [1402, 0.5096897]]
S: [[2088, 0.3751691], [3201, 0.37633234], [2739, 0.38411123], [1960, 0.39031777], [3295, 0.42794657]]
S: [[2088, 0.3751691], [3201, 0.37633234], [2739, 0.38411123], [1960, 0.39031777], [3295, 0.427946

In [26]:
# Try for a silly comparison just to prove everything works as we might hope.
# generated from https://randomwordgenerator.com/sentence.php
sillysentences = [
     "I know many children ask for a pony, but I wanted a bicycle with rockets strapped to it."
    ,"The swirled lollipop had issues with the pop rock candy."
    ,"As the years pass by, we all know owners look more and more like their dogs."
    ,"When nobody is around, the trees gossip about the people who have walked under them."
    ,"She saw the brake lights, but not in time."
    ,"Karen realized the only way she was getting into heaven was to cheat."
    ,"""
It’s difficult to reflect on the past year—or forecast the next 
without a sense of wonder regarding the sheer magnitude of innovation taking place 
across the AI landscape. 
On a weekly basis, researchers across industry and academia have published work 
advancing the state-of-the-art in nearly every domain of AI, toppling benchmarking 
leaderboards and accomplishing feats beyond what we could have imagined even a few years ago.
""" # <--- not silly
    ,"""
Networking firm Cisco Systems Inc. will add Microsoft Corp.'s 
Teams messaging app to its meeting devices, the two firms said on Wednesday, 
offering users an alternative to its own Webex video conferencing app.    
""" # <--- not silly
    ,"""
Apple CEO Tim Cook confirmed that Apple will buy U.S.-made microchips at an event in Arizona 
on Tuesday, where President Joe Biden also spoke.
Cook said Apple would buy processors made in a new Arizona factory, 
according to a video from the event.The chip factories will be owned 
and operated by Taiwan Semiconductor Manufacturing Company, 
the biggest foundry company with over half of the global market share.
TSMC produces the most advanced processors, including the chips in the latest 
iPhones, iPads and Macs.
""" # <--- not silly
]

sillysentence_embeddings = model.encode(sillysentences)
sillysentence_embeddings.shape

(9, 384)

In [27]:
scores = []
for i in range (0, len(sillysentence_embeddings)):
    cs = cosine_similarity(
        [sillysentence_embeddings[i]],
        cSentence_embeddings[:]
        )
    s = getHiFive(cs[0])
    scores.append(s)
    
complete(state="silly estimations")    

S: [[2211, 0.2276538], [46, 0.23272768], [893, 0.24209307], [3072, 0.24409437], [1423, 0.2654419]]
S: [[2401, 0.26178062], [3423, 0.2645874], [1633, 0.26976463], [1985, 0.2796421], [2402, 0.30769524]]
S: [[3497, 0.256158], [3537, 0.2565754], [3503, 0.2658984], [1642, 0.27100718], [2979, 0.28025562]]
S: [[2317, 0.2063945], [2309, 0.21671245], [3076, 0.22479649], [1629, 0.22780573], [3409, 0.27146274]]
S: [[3444, 0.27391848], [2207, 0.2787892], [859, 0.29351252], [2383, 0.3020659], [2038, 0.30933064]]
S: [[2427, 0.20280904], [249, 0.2034355], [681, 0.20654726], [2203, 0.22711846], [2200, 0.23493478]]
S: [[2505, 0.60572565], [2717, 0.6136384], [1962, 0.6146476], [2297, 0.61497074], [400, 0.6589467]]
S: [[2925, 0.503388], [1622, 0.50824255], [1895, 0.52069104], [2384, 0.5207633], [1191, 0.54956734]]
S: [[2322, 0.53968096], [1130, 0.5466041], [150, 0.5600075], [3221, 0.5682373], [1793, 0.5699532]]

Cell silly estimations @ 2023-02-07 22:05:13


In [28]:
# Reverse the list
revScore = []
print (f"Total Scores : {len(scores)}")
for score in scores:
    score = score[::-1]
    print(f"Score:{score}")
    revScore.append(score)

Total Scores : 9
Score:[[1423, 0.2654419], [3072, 0.24409437], [893, 0.24209307], [46, 0.23272768], [2211, 0.2276538]]
Score:[[2402, 0.30769524], [1985, 0.2796421], [1633, 0.26976463], [3423, 0.2645874], [2401, 0.26178062]]
Score:[[2979, 0.28025562], [1642, 0.27100718], [3503, 0.2658984], [3537, 0.2565754], [3497, 0.256158]]
Score:[[3409, 0.27146274], [1629, 0.22780573], [3076, 0.22479649], [2309, 0.21671245], [2317, 0.2063945]]
Score:[[2038, 0.30933064], [2383, 0.3020659], [859, 0.29351252], [2207, 0.2787892], [3444, 0.27391848]]
Score:[[2200, 0.23493478], [2203, 0.22711846], [681, 0.20654726], [249, 0.2034355], [2427, 0.20280904]]
Score:[[400, 0.6589467], [2297, 0.61497074], [1962, 0.6146476], [2717, 0.6136384], [2505, 0.60572565]]
Score:[[1191, 0.54956734], [2384, 0.5207633], [1895, 0.52069104], [1622, 0.50824255], [2925, 0.503388]]
Score:[[1793, 0.5699532], [3221, 0.5682373], [150, 0.5600075], [1130, 0.5466041], [2322, 0.53968096]]


In [ ]:
print(scores[0])
print(revScore[0])

### Test Results
From the tests above, it seems clear that the estimation method works. One question is where we do the cutoffs?

Results in the 0.0 to 0.65x range seem to be below cutoff.

Results in the 0.7 and above range seem to be right for inclusion,

But what about something in the range 0.6 to 0.699?

### Edited on 01/24/2023
Switched transformer model from 'bert-base-nli-mean-tokens' to the model used to obtain HF scores: 'all-MiniLM-L6-v2'

New scores are much lower than seen previously, more similar to HF score range

In [ ]:
# write out the KeyBERT Scores and add to candidate score df 

In [ ]:
xSummaries.head()

In [ ]:
print(revScore[0])

revScore[0][1][1]

In [ ]:
# create df out of KeyBERT score list
cols = list('abcde')
dftest = pd.DataFrame(revScore, columns=cols)
dftest

In [ ]:
# demonstrate can flatten column and isolate score
dftest1 = pd.DataFrame(dftest['a'].values.tolist(), columns=['a','b'])
dftest1 = dftest1.drop(columns=['a'])
dftest1

In [ ]:
#iterate through scores df, expand columns, then add KB score column to xSummaries df
scoreIT = 1

for column in dftest:
    col_name = 'kb'+ str(scoreIT)
    
    tempdf = pd.DataFrame(dftest[column].values.tolist(), columns=['a','b'])
    xSummaries[col_name] = tempdf['b']
    
#     print(col_name)
    scoreIT += 1

In [ ]:
# preview modified candidate df with hf and kb scores
pd.set_option("max_colwidth", 30)
pd.set_option("expand_frame_repr", True)
xSummaries.head()

### Calculate weighted average of KB scores:
Use same method as used for HF score average, remove outliers first using IQR, then find the mean

In [ ]:
def removeOutliers(test_list):
    q1 = np.percentile(test_list, 25)
    q3 = np.percentile(test_list, 75)
    iqr = (q3 - q1) * 1.5
    q_set = (q1 - iqr, q3 + iqr)

    result_list =[]

    for num in test_list:
        if num >= q_set[0] and num <= q_set[1]:
            result_list.append(num)

    return result_list

In [ ]:
def calculateScoreAvg(df):
    score_avg = []
    
    for i in range(len(df)):
        row_list = df.loc[i, ['kb1', 'kb2', 'kb3', 'kb4', 'kb5']].values.flatten().tolist()
        test_list = list(map(float, row_list))
        
        new_list = removeOutliers(test_list)
        weighted_score = np.mean(new_list)
#         print(f"Weighted score of row {i}: {weighted_score}")
        
        score_avg.append(weighted_score)
        i +=1
        
    return score_avg

In [ ]:
kb_avg = calculateScoreAvg(xSummaries)

xSummaries['kbAvg'] = kb_avg
xSummaries

In [ ]:
xSummaries.to_json(r'/home/deleidos/Notebooks/JSONData/semanticSimArticlesTestingDF.json')
complete(state="updates written: ")

In [ ]:
# cArticleURLs = cSummaries.url.to_list()
# # print ( f"scores[0]: {scores[0]}")
# for score in scores[0]:
#     articleID = score[0]
#     print (f"Score: {articleID} URL most similar: {cArticleURLs[articleID]}")

In [ ]:
# print (scores[0][0][0])
# print (f" before: {xSummaries.iloc[0]}")
# xSummaries.iloc[0]['KB1'] = scores[0][0][0]
# DF2 = DF2 = pd.DataFrame([[11,12,13,14]], columns = ('x', 'y', 'z', 'u'), index=['A']) 
# xSummaries.iloc[0][['KB1', 'KB2', 'KB3', 'KB4'] ] = DF2[['x', 'y', 'z', 'u']]
